In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun May  2 23:57:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
import datetime as dt
import numpy as np
import re
import os
import random
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8567186409167329728
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15703311680
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8324733082423217793
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [4]:
#Change based on devices available
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
sess = tf.compat.v1.Session(config=config) 
tf.compat.v1.keras.backend.set_session(sess)

In [5]:
#Run only once
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
scripts_dir = 'scripts/'#'Scripts/Scripts/'
scripts = os.listdir(scripts_dir)
# random.shuffle(scripts)
if '.ipynb_checkpoints' in scripts:
  scripts.remove('.ipynb_checkpoints')
print(scripts)
#Creating Neural Network
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    #50 neurons wont be best results, but will be fast
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256))
    # model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
              
    model.summary()
              
    return model

['Episode3RevengeOfTheSith.txt', 'Episode2AttackOfTheClones.txt', 'Episode1ThePhantomMenace.txt', 'Episode7TheForceAwakens.txt', 'Episode5TheEmpireStrikesBack.txt', 'Episode4NewHope.txt', 'Episode6ReturnOfTheJedi.txt']


In [7]:
def cleanup(line):
    # line = line.replace(r'<[^>]+>', ' ')
    # line = line.replace('<b>',' <\n> ')
    # line = line.replace('</b>',' <\n> ')
    # line = line.replace('<br>',' \n ')
    line = re.sub(r"<b>",' BREAKLINE1 ', line)
    line = re.sub(r"</b>",' BREAKLINE ', line)
    line = re.sub(r"<br>",' BREAKLINE ', line)

    line = re.sub(r"[^\w'()\s]",' ', line)
    line = re.sub(r'OBI WAN', 'OBI-WAN', line)
    line = re.sub(r'Obi wan', 'Obi-wan', line)
    line = re.sub(r'QUI GON', 'QUI-GON', line)
    line = re.sub(r'Qui gon', 'Qui-gon', line)
    return line

def preprocess(scripts, n_scripts=8, train_len=20):
    train_len += 1
    
    cleaned_script_sequences = []
    
    count = 0   
    t = dt.datetime.now()
    for script in scripts[:n_scripts]:
        count += 1
        with open(scripts_dir+script, "r", encoding="utf8") as f:
            tokens = []
            sequences = []
            for line in f:
                line = cleanup(line)#lower?
                # line = word_tokenize(line)
                clean_tok = []
                for tok in re.split('\s', line):
                  if tok != '':
                    clean_tok.append(tok)
                tokens.extend(clean_tok)
                
                #Progress print
                if (dt.datetime.now() -t).seconds / 10 >= 1:
                    t = dt.datetime.now()
                    percent = count/n_scripts
                    print('\rScripts Preprocessing: {:.2%} Done'.format(percent), end='', flush=True)
                    
            for i in range(train_len, len(tokens)):
                seq = tokens[i-train_len: i]
                sequences.append(seq)
                
                #Remove later
                cleaned_script_sequences.append(seq)
            # cleaned_script_sequences.append(sequences)
    
    print('\rScripts Preprocessing: 100% Done \n', end='', flush=True)
    return cleaned_script_sequences
  
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_Words):
    
    output_text = []
    
    #Seed text we feed in
    input_text = seed_text
    
    for i in range(num_gen_Words):
        #Tokenizes the input seed text
        encode_text = tokenizer.texts_to_sequences([input_text])[0]
        
        #padding seed text to fit the sequence length, 
        #and truncating the begining if seed is longer than seqence length
        pad_encoded = pad_sequences([encode_text], maxlen=seq_len, truncating='pre')
        
        #Predict next word using the padded_encoded seed text
        #Grabs first index
        pred_word_ind = np.argmax(model.predict(pad_encoded), axis=-1)[0]
        
        #Gets the word from the tokenizer
        pred_word = tokenizer.index_word[pred_word_ind]
        
        #Update input_text
        input_text += ' '+pred_word
        
        #Adding predicted word to output text
        output_text.append(pred_word)
    
        final = ' '.join(output_text)
        
        final = re.sub('BREAKLINE1',' \n\n',final)
        final = re.sub('BREAKLINE',' \n ',final)
        
    return final

processed = preprocess(scripts)
random_pick = random.randint(0, len(processed))
random_seed_text = processed[random_pick]
print(random_seed_text)

Scripts Preprocessing: 100% Done 
['frantically', 'back', 'over', 'his', 'shoulder', 'at', 'Artoo', 'BREAKLINE1', 'EXT', "LUKE'S", 'X', 'WING', 'FIGHTER', 'BREAKLINE', 'Smoke', 'billows', 'out', 'around', 'little', 'Artoo', 'and']


In [8]:
print("Fiting text")
tokenizer = Tokenizer(filters='!"#$%&*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=False)
tokenizer.fit_on_texts(processed)
sequences = tokenizer.texts_to_sequences(processed)

vocabulary_size = len(tokenizer.word_counts)
sequences = np.array(sequences)

#Take everything except last word (column)
X = sequences[:,:-1]

#Take last word (column)
y = sequences[:,-1]

print("y to categorical")
y = to_categorical(y, num_classes=vocabulary_size+1)
seq_len = X.shape[1]

Fiting text
y to categorical


In [9]:
print("creating model")
model = create_model(vocabulary_size+1, seq_len)

# from pickle import dump, load
model.fit(X,y, batch_size=256, epochs=300, verbose=1)

creating model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 20)            298400    
_________________________________________________________________
lstm (LSTM)                  (None, 20, 256)           283648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 14920)             3834440   
Total params: 5,007,592
Trainable params: 5,007,592
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
823/823 [==============================] - 27s 25ms/step - loss: 7.3217 - accuracy: 0

In [14]:
print("random seed")
random_pick = random.randint(0, len(processed))
random_seed_text = processed[random_pick]
seed_text = ' '.join(random_seed_text)


print(f'Seed Text: \n{seed_text}\n')

new_text = generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_Words=300)

print(f'\nNew Text: \n{new_text}')

random seed
Seed Text: 
those kids were I did everything I could to save him but he died they all did They were never able


New Text: 
to adapt to live out of here CAPT TARPALS said Jedi destroy him QUI-GON Everything do to Naboo dare looks on the Outer side and concentrates points hands steady unintelligible They excited pieces flying to you  
   
  ANAKIN What are  
   
  OBI-WAN BALL He requests me can you all  
   
   
  As PALPATINE rushes to the veranda  
   
  ANAKIN (continuing) we have the Chosen of the Ancient Order and I'm not like me Lord Plagueis reinforce the Federation's be request afraid You must be a problem Only Yes what I don't do it Yoda would have been a little election you Luke  
   
  ANAKIN I know  
   
  PALPATINE Why Why I did not real longer against this mystery  
   
  PADME I only sense you have not over a new honor quickly jumps and the war with the sovereign system of the pirateship  

 LUKE  
  I don't think I was right certainly vote Annie foolish as w

In [11]:
model.save('7Starwars-256-300_NOpunc-4.h5')
# dump(tokenizer, open('my_simple_tokenizer', 'wb'))

In [12]:
from google.colab import files
files.download('7Starwars-256-300_NOpunc-4.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
print(random_pick )

25266
